<a href="https://colab.research.google.com/github/LShahmiri/Computer_vision-Image_processing_Projects/blob/master/EMGMM_PCA/EMGMM_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import imageio
import cv2
from sklearn.decomposition import PCA
from scipy import stats

class ExpectationMaximizationGMM:

    def __init__(self, num_clusters):
        self.num_clusters = num_clusters

    def fit(self, X):
        num_clusters = self.num_clusters
        mean, covariance, weights, mixture_matrix = self.initialize_parameters(num_clusters)
        tolerance = 0.001
        max_iterations = 200
        self.log_likelihoods = []

        old_likelihood = self.calculate_log_likelihood(mixture_matrix)

        for iteration in range(max_iterations):
            sys.stdout.write('\r' + f'Number of clusters: {num_clusters}, Iteration: {iteration}')
            gamma = self.calculate_gamma(mixture_matrix)
            Nk = self.calculate_Nk(gamma)
            mean = self.calculate_mean(X, gamma, Nk)
            covariance = self.calculate_covariance(X, gamma, Nk, mean)
            weights = self.calculate_weights(Nk, X.shape[0])

            mixture_matrix = self.calculate_gaussian_mixture(X, mean, covariance, weights)
            current_likelihood = self.calculate_log_likelihood(mixture_matrix)
            self.log_likelihoods.append(current_likelihood)

            if current_likelihood - old_likelihood < 0:
                print("\nThere is a problem!")
                return
            elif current_likelihood - old_likelihood < tolerance:
                break

            old_likelihood = current_likelihood

        print("\nSuccessful")
        self.predicted_X = mean[gamma.argmax(axis=1)]

# Set your root directories
root_directory_input = "/content/drive/MyDrive/Datasets/---/"
root_directory_output_emgmm = '/content/drive/MyDrive/Datasets/---/1/'
root_directory_output_exception = '/content/drive/MyDrive/Datasets/----/2/'

# Loop through folders and images
for folder in os.listdir(root_directory_input):
    directory_output_emgmm = os.path.join(root_directory_output_emgmm, folder)
    directory_input = os.path.join(root_directory_input, folder)

    if not os.path.exists(directory_output_emgmm):
        os.mkdir(directory_output_emgmm)

    for file_path in glob.glob(str(directory_input) + '/*.*'):
        filename = os.path.basename(file_path)
        image_original = imageio.imread(file_path)
        shape = image_original.shape

        try:
            emgmm = ExpectationMaximizationGMM(4)

            # Apply EMGMM on the original color image
            emgmm.fit(image_original.reshape((-1, 3)) / 255)

            # PCA Dimensionality Reduction for each color channel
            pca = PCA(n_components=0.95, whiten=True)
            reduced_r = pca.fit_transform(image_original[:, :, 0])
            reduced_g = pca.fit_transform(image_original[:, :, 1])
            reduced_b = pca.fit_transform(image_original[:, :, 2])

            # Inverse PCA and Image Reconstruction
            reconstructed_r = pca.inverse_transform(reduced_r)
            reconstructed_g = pca.inverse_transform(reduced_g)
            reconstructed_b = pca.inverse_transform(reduced_b)

            reconstructed_image = np.stack((reconstructed_r, reconstructed_g, reconstructed_b), axis=-1)
            result = np.clip((reconstructed_image * 255).astype(np.uint8), 0, 255)

            # Save Processed Images
            output_path_emgmm = os.path.join(directory_output_emgmm, filename)
            imageio.imwrite(output_path_emgmm, result)

        except Exception as e:
            directory_output_exception = os.path.join(root_directory_output_exception, folder)
            if not os.path.exists(directory_output_exception):
                os.mkdir(directory_output_exception)
            output_path_exception = os.path.join(directory_output_exception, filename)
            imageio.imwrite(output_path_exception, image_original)
